In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=e49c126db72e2734e7d602bee9c0353ac773e94a862fc08492c7aad27f40fcae
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-jqqoauz9/kobert-tokenizer_29601c9165b540c4b02425cf28e47732
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-jqqoauz9/kobert-tokenizer_29601c9165b540c4b02425cf28e47732
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=a282a5f661e5a09c5e47250a348f3ce080dccb0c41f6d080ea9b77b336feae6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-cb1qagdq/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# GPU 사용 시
device = torch.device("cuda:0")

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/캡스톤/data/kakao_final_data.csv')
df1.head()

,date,time,title,content,Date,Open,High,Low,Close,Volume,Change,score
0,2022.01.03,04:31:16,비운의 세대 집콕을 자기계발 기회 삼았죠 20학번의 코로나 극복기,캠퍼스 낭만 비켜 간 적막한 대학 생활 채팅과 SNS로 먼저 인연 맺은 동기들 학식...,2022.01.03,0.992464,1.0,1.0,1.0,0.068944,0.392385,-1.0
1,2022.01.03,03:03:34,전업 주부부터 검사까지 그들은 왜 웹소설에 빠졌나,웹소설 작가 20만명 시대 부부 작가팀 싱숑이 쓴 판타지 웹소설 전지적 독자 시점 ...,2022.01.03,0.992464,1.0,1.0,1.0,0.068944,0.392385,-1.0
2,2022.01.03,00:03:12,광고 모델 넘어 앵커 쇼호스트 은행원까지 가상 인간 실제 세상에 스며들다,가상 인간이 온다 로지 한국 2020 인스타그램 팔로워 11만명 국내 최초 가상 인...,2022.01.03,0.992464,1.0,1.0,1.0,0.068944,0.392385,-1.0
3,2022.01.03,09:02:04,4인 9시 거리두기 2주 연장 10일부터 백화점 마트도 방역패스,손실보상금 500만원 선지급 청소년 방역패스는 3월부터 오미크론 감염 하루에 269...,2022.01.03,0.992464,1.0,1.0,1.0,0.068944,0.392385,-1.0
4,2022.01.03,00:21:28,대기업 빅테크 사모펀드 합종연횡 빅 블러 심화될 것,새해 M A 시장 전망 83조원 지난해 국내 인수합병 M A 시장 규모다 2012년...,2022.01.03,0.992464,1.0,1.0,1.0,0.068944,0.392385,-1.0


In [10]:
df2 = pd.read_csv('/content/drive/MyDrive/캡스톤/data/crafton_final_data.csv')
df2.head()

,date,time,title,content,Date,Open,High,Low,Close,Volume,Change,score
0,2022.01.03,18:21:01,최태원 기획 오디션서 대상 인생 가장 큰 고통서 힌트 얻었죠,대한상의 국가발전 프로젝트 대상에 사소한 형제들 치매진단 영상통화 사소한 통화 개발...,2022.01.03,1.0,1.0,1.0,1.0,0.033538,0.412413,-1
1,2022.01.03,18:02:03,몸값 1조 넘는 대어만 14곳 대기 공모액 30조 역대급 이어간다,IPO 호황 새해에도 계속 작년 총 20조4500억 기록 이어 이달 LG엔솔 흥행 ...,2022.01.03,1.0,1.0,1.0,1.0,0.033538,0.412413,-1
2,2022.01.03,16:56:05,새해에도 IPO 시장 후끈,LG에너지솔루션 차량용 배터리 탑재 전기차 연합뉴스 지난해 역대급 을 기록한 기업공...,2022.01.03,1.0,1.0,1.0,1.0,0.033538,0.412413,-1
3,2022.01.03,17:36:01,이부진 이서현과 어깨 나란히 주식부호 톱5 들어간 창업주는,1년새 5조원 늘어 14조원 지난해 국내 증시 시가총액이 전년에 비해 283조원가량...,2022.01.03,1.0,1.0,1.0,1.0,0.033538,0.412413,-1
4,2022.01.03,17:42:01,배틀그라운드 모바일게임 매출 세계 1위,크래프톤의 배틀그라운드 모바일 배그 모바일 이 지난해 글로벌 모바일 게임 매출 1위...,2022.01.03,1.0,1.0,1.0,1.0,0.033538,0.412413,-1


In [11]:
df3 = pd.read_csv('/content/drive/MyDrive/캡스톤/data/naver_final_data.csv')
df3.head()

,date,time,title,content,Date,Open,High,Low,Close,Volume,Change,score
0,2022.01.03,03:03:34,전업 주부부터 검사까지 그들은 왜 웹소설에 빠졌나,웹소설 작가 20만명 시대 부부 작가팀 싱숑이 쓴 판타지 웹소설 전지적 독자 시점 ...,2022.01.03,1.0,1.0,1.0,1.0,0.026326,0.433827,-1.0
1,2022.01.03,00:21:28,대기업 빅테크 사모펀드 합종연횡 빅 블러 심화될 것,새해 M A 시장 전망 83조원 지난해 국내 인수합병 M A 시장 규모다 2012년...,2022.01.03,1.0,1.0,1.0,1.0,0.026326,0.433827,-1.0
2,2022.01.03,09:02:04,4인 9시 거리두기 2주 연장 10일부터 백화점 마트도 방역패스,손실보상금 500만원 선지급 청소년 방역패스는 3월부터 오미크론 감염 하루에 269...,2022.01.03,1.0,1.0,1.0,1.0,0.026326,0.433827,-1.0
3,2022.01.03,04:51:01,상표 도용 아바타 성추행 새해 메타버스 범죄 판친다,발 사진 찍어서 보내줄 수 있어 초등학생 A양 11 이 지난해 11월 확장 가상세계...,2022.01.03,1.0,1.0,1.0,1.0,0.026326,0.433827,-1.0
4,2022.01.03,17:07:02,삼성전자 십만전자 가능 현대차 네이버 기세등등,2022 주식투자 가이드 올해의 유망 주식 한국 현대차 네이버 기세등등 사진 뉴스1...,2022.01.03,1.0,1.0,1.0,1.0,0.026326,0.433827,-1.0


In [12]:
df4 = pd.read_csv('/content/drive/MyDrive/캡스톤/data/ncsoft_final_data.csv')
df4.head()

,date,time,title,content,Date,Open,High,Low,Close,Volume,Change,score
0,2022.01.03,16:40:04,NFT 거래는 남의 일 생활 곳곳서 퍼진다 2022 신년기획,V KOREA 디지털 시대 미래기술이 온다 NFT 기반 디지털 소유권 없인 메타버스...,2022.01.03,0.94963,1.0,0.970194,1.0,0.341744,0.587173,0.0
1,2022.01.03,17:11:01,이름만 사소한 통화 치매 가족 살릴 위대한 아이디어,대한상의 국가발전 프로젝트 결과 이봉주씨 치매 예방 아이템 1위 상금 1억 선정 대...,2022.01.03,0.94963,1.0,0.970194,1.0,0.341744,0.587173,0.0
2,2022.01.03,19:50:09,P2E를 장악하라 토종업계 블록체인 게임 전쟁,위메이드 P2E 분야 선두주자 컴투스 서머너즈 워 등 10종 넷마블 NC도 신작에 ...,2022.01.03,0.94963,1.0,0.970194,1.0,0.341744,0.587173,0.0
3,2022.01.03,10:40:22,P2E 선두주자 위메이드 올해 100개 서비스 게임도 하고 돈도 번다,위메이드 미르4 동시접속자 130만명 글로벌 시장 가능성 확인 오픈 게이밍 블록체인...,2022.01.03,0.94963,1.0,0.970194,1.0,0.341744,0.587173,0.0
4,2022.01.03,19:52:07,북미 유럽시장 잡자 콘솔에 눈돌린 게임사,붉은사막 펄어비스 제공 국내 게임사들이 올해 PC와 모바일을 넘어 콘솔 게임 영역으...,2022.01.03,0.94963,1.0,0.970194,1.0,0.341744,0.587173,0.0


In [13]:
def set_data(df):
  df = df1[['date', 'title', 'score']]
  df['score'] = df['score'].astype(float)
  df['title'] = df['title'].astype(str)
  grouped_data = df.groupby('date').agg({'title': ' '.join, 'score': 'mean'})
  return grouped_data

In [14]:
grouped_data1 = set_data(df1)

<ipython-input-13-50d121f20685>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df['score'].astype(float)
<ipython-input-13-50d121f20685>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)


In [15]:
grouped_data2 = set_data(df2)

<ipython-input-13-50d121f20685>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df['score'].astype(float)
<ipython-input-13-50d121f20685>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)


In [16]:
grouped_data3 = set_data(df3)

<ipython-input-13-50d121f20685>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df['score'].astype(float)
<ipython-input-13-50d121f20685>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)


In [17]:
grouped_data4 = set_data(df4)

<ipython-input-13-50d121f20685>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df['score'].astype(float)
<ipython-input-13-50d121f20685>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)


In [18]:
grouped_data = pd.concat([grouped_data1, grouped_data2, grouped_data3, grouped_data4], axis = 0)
grouped_data.head()

,title,score
date,,
2022.01.03,비운의 세대 집콕을 자기계발 기회 삼았죠 20학번의 코로나 극복기 전업 주부부터 검...,-1.0
2022.01.04,카카오 김범수 의장 중심 미래사업 컨트롤타워 구성 조건 없이 3% 적립 카카오페이신...,-1.0
2022.01.05,베일 벗은 카카오의 미래전략 김범수 MBN 으로 승부수 띄웠다 카카오에 밀리고 우티...,-1.0
2022.01.06,카카오게임즈 하루새 14% 급락 리더십 위기빠진 류영준 카카오 노조 대표 선임 철회...,0.0
2022.01.07,카카오엔터 웹툰 나 혼자만 레벨업 NFT 발행 시총 30 날아간 카카오 엔터 IPO...,-1.0


In [19]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984 entries, 2022.01.03 to 2022.12.29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   984 non-null    object 
 1   score   984 non-null    float64
dtypes: float64(1), object(1)
memory usage: 23.1+ KB


In [20]:
from sklearn.model_selection import train_test_split

train1, test1 = train_test_split(grouped_data, test_size=0.2, random_state=123)

In [21]:
train1.head()

,title,score
date,,
2022.09.02,금리 비싸도 편해서 인터넷은행 3사 대출 8개월째 늘어 40조 돌파 디지털플랫폼정부...,0.0
2022.07.04,요즘 누가 적금 드냐 는 말 사라졌다 MZ들이 하니까 이 앱은 안전한 결제시스템을 ...,1.0
2022.01.21,주식 먹튀 논란 카카오 경영진 대폭 물갈이 서울제로페이 신한카드 카카오 품에 안겼다...,-1.0
2022.05.18,카카오 ESG 경영 최세정 사외이사가 이끈다 네이버 카카오 사회적 가치 지향 임팩트...,-1.0
2022.09.13,네이버 카카오는 두 창업자의 국감 증인 출석 막아낼 수 있을까 교통약자 이동 콘텐츠...,-1.0


In [22]:
test1.head()

,title,score
date,,
2022.03.25,880억 성과급 달라 카카오 임지훈 전 대표 김범수 카벤에 소송 시총 2위로 보는 ...,0.0
2022.03.04,네이버 vs 카카오 라방 해외시장서 격돌 서울사랑상품권 플랫폼 분쟁 서울시 판정승 ...,-1.0
2022.03.03,고객 인재 몰리는 인터넷은행 서울대병원이 권하는 슬기로운 재택치료 카톡서 손쉽게 확...,-1.0
2022.07.07,공은 방통위로 구글 카카오 삼자대면 MBK 카카오모빌리티 지분 50% 1주 4조에 ...,-1.0
2022.09.29,네이버 카카오 자체 서제스트 주력 서비스 고도화 바닥 뚫은 카카오금융주 뱅크 페이 ...,1.0


In [23]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, 2022.09.02 to 2022.01.27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   787 non-null    object 
 1   score   787 non-null    float64
dtypes: float64(1), object(1)
memory usage: 18.4+ KB


In [24]:
train1 = train1.dropna()

In [25]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, 2022.09.02 to 2022.01.27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   787 non-null    object 
 1   score   787 non-null    float64
dtypes: float64(1), object(1)
memory usage: 18.4+ KB


In [26]:
# 0을 중립 1은 긍정 2는 부정
test1['score'] = test1['score'].apply(lambda x: 2 if x == -1 else x)
train1['score'] = train1['score'].apply(lambda x: 2 if x == -1 else x)

In [27]:
train1 = train1[train1.score != -2.0]

In [28]:
train1.groupby('score').count()

,title
score,
0.0,281
1.0,208
2.0,294


In [29]:
test1.groupby('score').count()

,title
score,
0.0,71
1.0,56
2.0,70


In [30]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 783 entries, 2022.09.02 to 2022.01.27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   783 non-null    object 
 1   score   783 non-null    float64
dtypes: float64(1), object(1)
memory usage: 18.4+ KB


In [31]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197 entries, 2022.03.25 to 2022.07.12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   197 non-null    object 
 1   score   197 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.6+ KB


In [32]:
test1 = test1[test1.score != -2]

In [33]:
test1.head()

,title,score
date,,
2022.03.25,880억 성과급 달라 카카오 임지훈 전 대표 김범수 카벤에 소송 시총 2위로 보는 ...,0.0
2022.03.04,네이버 vs 카카오 라방 해외시장서 격돌 서울사랑상품권 플랫폼 분쟁 서울시 판정승 ...,2.0
2022.03.03,고객 인재 몰리는 인터넷은행 서울대병원이 권하는 슬기로운 재택치료 카톡서 손쉽게 확...,2.0
2022.07.07,공은 방통위로 구글 카카오 삼자대면 MBK 카카오모빌리티 지분 50% 1주 4조에 ...,2.0
2022.09.29,네이버 카카오 자체 서제스트 주력 서비스 고도화 바닥 뚫은 카카오금융주 뱅크 페이 ...,1.0


In [34]:
test1 = test1.dropna()
train1= train1.dropna()

In [35]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197 entries, 2022.03.25 to 2022.07.12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   197 non-null    object 
 1   score   197 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.6+ KB


In [36]:
class BERTSentenceTransform:
    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

    def __call__(self, line):
       
        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
           
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
          
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        valid_length = len(input_ids)

        if self._pad:
          
            padding_length = self._max_seq_length - valid_length
           
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [37]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer ,vocab, max_len,
                 pad, pair):
   
        #transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab = vocab, pad=pad, pair=pair)
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab = vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [38]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [39]:
#tok = tokenizer.tokenize

data_train = BERTDataset(train1.values, 0, 1, tokenizer,  vocab, max_len, True, False)
data_test = BERTDataset(test1.values, 0, 1, tokenizer, vocab, max_len, True, False)

In [40]:
data_train[0]

(array([   2, 1237, 2514, 6752, 5859, 4832, 7850, 3794, 7087,  589, 6493,
        1666,  624, 5366, 7369, 1553,  606, 7253, 1739, 1832, 7761, 6030,
        7739, 7230, 7054,  517, 6418, 6356, 1143, 2358,  811, 4555, 3397,
        4955, 5561, 7227, 2546, 7619, 2488, 7884, 4389, 4075,  627, 7028,
        2677, 2120, 7611, 1464, 2695, 5680, 5760, 1828, 7795,  517, 7517,
        6122, 1019, 7264,  659, 6321, 7565,  517,  303,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [41]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [42]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [43]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [44]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [46]:
train_history = []
test_history = []
loss_history = []
best_accuracy = 0.0
best_model = None
early_stop = 0

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length.to(device)  # valid_length에도 .to(device) 적용
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        train_batch_acc = calc_accuracy(out, label)  # 배치별 정확도 기록
        train_acc += train_batch_acc
        train_history.append(train_batch_acc.item())  # 배치별 정확도 저장
        loss_history.append(loss.item())

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(
                e+1, batch_id+1, loss.item(), train_batch_acc.item()))

    avg_train_acc = train_acc / len(train_dataloader)  # 평균 정확도 계산
    print("epoch {} train acc {}".format(e+1, avg_train_acc))

    model.eval()

    with torch.no_grad():
        test_acc = 0.0

        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length.to(device)
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)

            test_batch_acc = calc_accuracy(out, label)
            test_acc += test_batch_acc
            test_history.append(test_batch_acc.item())

        avg_test_acc = test_acc / len(test_dataloader)
        print("epoch {} test acc {}".format(e+1, avg_test_acc))
        test_history.append(avg_test_acc.item())

        # 성능이 가장 좋을 때 모델 저장
        if avg_test_acc > best_accuracy:
            best_accuracy = avg_test_acc
            best_model = model.state_dict()
            early_stop = 0

        elif early_stop == 3:
          break

        else:
          early_stop += 1

# 가장 좋은 모델 저장
torch.save(best_model, 'best_model.pth')

  8%|▊         | 1/13 [00:01<00:20,  1.68s/it]

epoch 1 batch id 1 loss 1.146083116531372 train acc 0.3125


100%|██████████| 13/13 [00:08<00:00,  1.54it/s]


epoch 1 train acc 0.3751602564102564


100%|██████████| 4/4 [00:00<00:00,  5.77it/s]


...

epoch 18 batch id 1 loss 0.09961973130702972 train acc 0.953125


100%|██████████| 13/13 [00:08<00:00,  1.59it/s]


epoch 18 train acc 0.9819711538461539


100%|██████████| 4/4 [00:00<00:00,  4.97it/s]


epoch 18 test acc 1.0


In [170]:
'''# 학습 코드
train_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        train_batch_acc = calc_accuracy(out, label)
        train_acc += train_batch_acc
        train_history.append(train_batch_acc.item())
        loss_history.append(loss.item())

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(
                e+1, batch_id+1, loss.item(), train_batch_acc.item()))

    avg_train_acc = train_acc / len(train_dataloader)
    print("epoch {} train acc {}".format(e+1, avg_train_acc))


# 테스트 코드
test_history = []

model.eval()

with torch.no_grad():
    test_acc = 0.0

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        test_batch_acc = calc_accuracy(out, label)
        test_acc += test_batch_acc
        test_history.append(test_batch_acc.item())

    avg_test_acc = test_acc / len(test_dataloader)
    print("epoch {} test acc {}".format(e+1, avg_test_acc))'''

'# 학습 코드\ntrain_history = []\nloss_history = []\n\nfor e in range(num_epochs):\n    train_acc = 0.0\n    model.train()\n\n    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):\n        optimizer.zero_grad()\n        token_ids = token_ids.long().to(device)\n        segment_ids = segment_ids.long().to(device)\n        valid_length = valid_length.to(device)\n        label = label.long().to(device)\n        out = model(token_ids, valid_length, segment_ids)\n\n        loss = loss_fn(out, label)\n        loss.backward()\n        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)\n        optimizer.step()\n        scheduler.step()\n\n        train_batch_acc = calc_accuracy(out, label)\n        train_acc += train_batch_acc\n        train_history.append(train_batch_acc.item())\n        loss_history.append(loss.item())\n\n        if batch_id % log_interval == 0:\n            print("epoch {} batch id {} loss {} train acc {}".format(\n